In [ ]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

# Load the model
model_dict = pickle.load(open("./model.p", "rb"))
model = model_dict["model"]

# MediaPipe setup
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Labels dictionary
labels_dict = {0: "1", 1: "2", 2: "3"}

# Specify image path directly here
image_path = r"C:\Repositories\Sign-Language-Detection\data\2\3.jpg"  # CHANGE THIS TO YOUR IMAGE PATH

# Read the image
image = cv2.imread(image_path)
if image is None:
    print(f"Error loading image: {image_path}")
    exit()

# Get image dimensions
H, W, _ = image.shape

# Convert to RGB for MediaPipe
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process the image
results = hands.process(image_rgb)

# Prepare data storage
data_aux = []
x_ = []
y_ = []

# Process hand landmarks if detected
if results.multi_hand_landmarks:
    # Draw landmarks
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style(),
        )
    
    # Process the first detected hand only
    hand_landmarks = results.multi_hand_landmarks[0]
    
    # First collect all coordinates
    for i in range(len(hand_landmarks.landmark)):
        x = hand_landmarks.landmark[i].x
        y = hand_landmarks.landmark[i].y
        x_.append(x)
        y_.append(y)
    
    # Normalize feature vector using min and max values
    x_min, x_max = min(x_), max(x_)
    y_min, y_max = min(y_), max(y_)
    for i in range(len(hand_landmarks.landmark)):
        x = hand_landmarks.landmark[i].x
        y = hand_landmarks.landmark[i].y
        data_aux.append((x - x_min) / (x_max - x_min))
        data_aux.append((y - y_min) / (y_max - y_min))

    # Debugging outputs
    print(f"Feature vector: {data_aux}")
    print(f"Feature vector length: {len(data_aux)}")
    
    # Draw bounding box
    x1 = int(min(x_) * W) - 10
    y1 = int(min(y_) * H) - 10
    x2 = int(max(x_) * W) + 10
    y2 = int(max(y_) * H) + 10
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Make prediction
    prediction = model.predict([np.asarray(data_aux)])
    predicted_character = labels_dict[int(prediction[0])]
    
    # Add prediction text
    cv2.putText(
        image,
        f"Prediction: {predicted_character}",
        (x1, y1 - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9,
        (0, 0, 255),
        2,
        cv2.LINE_AA,
    )
    
    # Print debug info
    print(f"Image: {image_path}")
    print(f"Prediction: {predicted_character}")
    print(f"Feature vector length: {len(data_aux)}")
    
    # Display the image
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"Prediction: {predicted_character}")
    plt.axis('off')
    plt.show()
else:
    print(f"No hand detected in {image_path}")
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("No hand detected")
    plt.axis('off')
    plt.show()